In [11]:
#https://github.com/minimaxir/facebook-page-post-scraper/blob/master/examples/how_to_build_facebook_scraper.ipynb
#https://towardsdatascience.com/how-to-use-facebook-graph-api-and-extract-data-using-python-1839e19d6999
#https://www.kdnuggets.com/2017/06/6-interesting-things-facebook-python.html

#!pip install requests 
import requests 
import urllib.request  as urllib2
import json
import datetime
import csv
import time
# Since the code output in this notebook leaks the app_secret,
# it has been reset by the time you read this.

app_id = "272535582777707"
app_secret = "59e7ab31b01d3a5a90ec15a7a45a5e3b" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

page_id = 'nytimes'

In [28]:
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = requests.get(url)
    print(base, " Response=>", req.status_code)
    response = req ; #urllib2.urlopen(req)
    data = req.json ;
    
    print (data ) 
    #.dumps(data, indent=4, sort_keys=True))
    

testFacebookPageData(page_id, access_token)

https://graph.facebook.com/v2.4  Response=> 400
<bound method Response.json of <Response [400]>>


In [33]:
def request_until_succeed(url):
    success = False
    c = 0 ;
    while success is False:
        try: 
            c += 1
            if ( c > 3 ):
                break 
            response = requests.get(url)
            if response.status_code == 200:
                success = True
        except e:
            print (e)
            time.sleep(10)
            
            print("Error for URL {}: {}" % (url, datetime.datetime.now()))

    return response


In [34]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = request_until_succeed(url)
    data = req.json ;
    
    print (data ) 

    
#    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageFeedData(page_id, access_token)

<bound method Response.json of <Response [400]>>


In [41]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    return request_until_succeed(url)
    
test_status = getFacebookPageFeedData(page_id, access_token, 1)
if ( test_status.status_code == 200 ):
    test_status = test_status['data']
    print (json.dumps(test_status, indent=4, sort_keys=True))
else:
    print(test_status)

<Response [400]>


In [ ]:
def processFacebookPageFeedStatus(status):
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print processed_test_status


In [ ]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)


scrapeFacebookPageFeedStatus(page_id, access_token)


In [ ]:
To get the groups he has joined:

API: /{user-id}/groups 
Permissions req: user_groups 


To get the pages he has liked:

API: /{user-id}/likes 
Permissions req: user_likes

In [ ]:
group_by(date = as.Date(substr(status_published, 1, 10))) %>%
                summarize(total_likes=sum(num_likes),
                          total_loves=sum(num_loves),
                          total_wows=sum(num_wows),
                          total_hahas=sum(num_hahas),
                          total_sads=sum(num_sads),
                          total_angrys=sum(num_angrys)) %>%
                arrange(date)

In [ ]:
gather(key=reaction, value=count, total_likes:total_angrys) %>%
                        mutate(reaction=factor(reaction))

print(head(df_agg_long,20))